# Motivation

In this notebook I'm looking at the difference between the modifiers in tumour derived networks. This is part of chapter 4 section 4.4 from my PhD thesis

# Init

In [1]:
%load_ext autoreload
import pandas as pd
import numpy as np
import os
import sys 

import plotly.express as px
import plotly.io as pio
from scipy import stats


import multiprocess as mp

sys.path.append('/Users/vlad/Developer/York/iNet_v2/src/')

from NetworkAnalysis.ExperimentSet import ExperimentSet
from NetworkAnalysis.NetworkComp import NetworkComp
from NetworkAnalysis import GraphHelper as gh
from NetworkAnalysis.utilities import sankey_consensus_plot as sky
from NetworkAnalysis.utilities.helpers import save_fig
import NetworkAnalysis.utilities.clustering as cs

pio.templates.default = "ggplot2"

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_base = "../../data/"
base_path = "../../results/"
exp_folder_tumour = "network_I/tum/"  # "/integration_v2.1/tum/" - path from iNET

# figures_path = base_path + exp_folder_tumour + "Figures/"
figures_path = "tum_modifiers/"

vu_output = pd.read_csv(f"{data_base}/metadata/VU_clustering_v3.tsv", sep="\t", index_col="Sample")

# prep mut
tcga_mutations_df = pd.read_csv(f"{data_base}/tumour/mutations_tcga.csv")
tcga_mutations_df = tcga_mutations_df[tcga_mutations_df["count"] != 0].set_index("gene")

all_tum_tpms = pd.read_csv(f"{data_base}/tumour/TPMs_selected_genes_v3_13k_gc42.tsv", sep="\t", index_col="genes")
all_tum_tpms_v4 = pd.read_csv(f"{data_base}/tumour/tum_TPMs_selected_genes_gc42_all_v4.tsv", sep="\t", index_col="genes")

# tf list
tf_path = f"{data_base}/metadata/TF_names_v_1.01.txt"
if os.path.exists(tf_path):
    tf_list = np.genfromtxt(fname=tf_path, delimiter="\t", skip_header=1, dtype="str")

## Load experiment sets

In [3]:
%autoreload 2

tum = ExperimentSet("tum", base_path, exp_folder_tumour, tcga_mutations_df, sel_sets = ["4K",], rel_path="../")
# p0 = ExperimentSet("p0", base_path, exp_folder_p0, tcga_mutations_df, sel_sets = ["4K"], rel_path="../")

%autoreload 2
tum.export_to_gephi(save=False)
# p0.export_to_gephi(save=False)

❗️ Check this experiment, there might be a problem in loading the data: norm2_int_tum_4K_50TF
##### Experiment labels:  dict_keys(['beta_4K_50TF', 'norm2_4K_50TF', 'standard_4K_50TF', 'beta2_4K_50TF', 'beta3_4K_50TF', 'norm3_4K_50TF', 'standard_4K_10TF', 'norm3_4K_10TF', 'beta_4K_10TF', 'standard_4K_6TF', 'beta_4K_6TF', 'norm3_4K_6TF', 'standard_4K_3TF', 'beta_4K_3TF', 'norm3_4K_3TF', 'standard_4K_5TF', 'standard_4K_8TF', 'standard_4K_4TF', 'standard_4K_7TF', 'beta_4K_5TF', 'beta_4K_4TF', 'norm3_4K_7TF', 'norm3_4K_5TF', 'beta_4K_8TF', 'norm3_4K_4TF', 'beta_4K_7TF', 'norm3_4K_8TF', 'standard_4K_9TF', 'norm3_4K_9TF', 'beta_4K_9TF'])


## Computed ModCon and MEV scores

In [4]:
def worker(arg):
    obj, methname = arg[:2]
    _ = getattr(obj, methname)()
    return obj

In [5]:
pool = mp.Pool(mp.cpu_count())

results = pool.map(worker, ((exp, "get_ModCon") for exp in tum.exps.values()))
tum.exps = {exp.type: exp for exp in results}

In [6]:
for exp in tum.get_exps():
    # exp.nodes_df["ModCon_Rank"] = 0
    for modCon, value in exp.modCons.items():
        dmy = value.sort_values(by=["ModCon_{}".format(exp.type)], ascending=False).reset_index(names="Id").iloc[:100]
        dmy["Rank"] = dmy.index + 1
        dmy.set_index("Id", inplace=True)
        exp.nodes_df.loc[exp.nodes_df["Modularity Class"] == modCon, "ModCon_Rank"] = dmy["Rank"]
        exp.nodes_df["ModCon_Rank"] = exp.nodes_df["ModCon_Rank"].fillna(0)

tum.export_to_gephi(save=False)
tum.generate_Mevs()

# Comms Mutation stats

In [7]:
std_exp = tum.exps["standard_4K_10TF"]

mut_genes = std_exp.mut_df[std_exp.mut_df["count"] > 0].index
expressed_genes = std_exp.tpm_df.index.values
len(mut_genes) / len(expressed_genes)

0.79975

In [8]:
dmy = []
sel_exp = tum.exps["norm3_4K_10TF"]
sort_col = "ModCon_{}".format(sel_exp.type)
for comm, value in sel_exp.modCons.items():
    modConGenes = value.sort_values(by=sort_col, ascending=False).iloc[:100]
    modConMutated = sel_exp.mut_df[sel_exp.mut_df.index.isin(modConGenes)].shape[0]
    dmy.append((comm, modConMutated / len(mut_genes)))

# Leiden score comparison

Looking at the leiden score comparison between experiments

In [9]:
leiden_scores = tum.comb_leiden_scores()

# the TF = 50 contain experiments with the different modifiers
leiden_scores = leiden_scores.loc[leiden_scores["TF"] != "50"]

# Specific to the tum dataset
leiden_scores.loc[leiden_scores["Modifier"] == "beta", "Modifier"] = "Penalty"

# Figure for multiple TFs and 3 Leiden scores
fig = px.scatter(leiden_scores, x="Modifier", color="TF", y="ModularityScore", size="ModuleNum", facet_col="Leiden Rank", facet_col_wrap=4)

# Fir for one TF and 10 Leiden scores
fig = gh.plot_leiden(exps=tum, tf="6")
fig.update_layout(font=dict(size=16))
# save_fig(name="LeidenMetrics_{}".format(label), fig=fig, base_path=figures_path, width=1400, height=600)

# Network Metrics

Between standard, norm and reward compare the network metrics: degree, pageRank, closeness, betwenees and IVI.

The network configuration is: 4K and 10TF. and 4K and 3TF.

## 4K 6TF (report)

In [ ]:
if True:
    std_nt, rwrd_nt, pen_nt = tum.exps["standard_4K_6TF"], tum.exps["norm3_4K_6TF"], tum.exps["beta_4K_6TF"]
    metrics_df = gh.prep_net_metrics(std_nt, rwrd_nt, pen_nt)

    fig = gh.plot_net_metrics(metrics_df, label="6TF", log_y=True, filename="NetworkMetricsComp_{}".format("6TF"), figs_path=figures_path)
    fig.show()

## Statistical tests (report)

In [11]:
from scipy import stats

dist_std = metrics_df.loc[metrics_df["Type"] == "Standard"]
dist_rwd = metrics_df.loc[metrics_df["Type"] == "Reward"]
dist_pen = metrics_df.loc[metrics_df["Type"] == "Penalised"]

In [10]:
metric = "degree"
statistic, p_value = stats.mannwhitneyu(
    dist_std[metric],
    dist_rwd[metric],
)
print(f"{metric}: Std vs Rwd Mann-Whitney U statistic: {statistic}, p-value: {p_value}")

statistic, p_value = stats.mannwhitneyu(
    dist_std[metric],
    dist_pen[metric],
)
print(f"{metric}: Std vs Pen Mann-Whitney U statistic: {statistic}, p-value: {p_value}")

######
one_degree_nodes = dist_pen[dist_pen["degree"] == 1].shape[0]
print("One degree nodes for penalised network ", one_degree_nodes)

degree: Std vs Rwd Mann-Whitney U statistic: 8062709.0, p-value: 0.5354114636615257
degree: Std vs Pen Mann-Whitney U statistic: 7864292.5, p-value: 0.981719202372621
One degree nodes for penalised network  39


In [12]:
# Closeness
metric = "closeness"
statistic, p_value = stats.mannwhitneyu(
    dist_std[metric],
    dist_rwd[metric],
)
print(f"{metric}: Std vs Rwd Mann-Whitney U statistic: {statistic}, p-value: {p_value}")

statistic, p_value = stats.mannwhitneyu(
    dist_std[metric],
    dist_pen[metric],
)
print(f"{metric}: Std vs Pen Mann-Whitney U statistic: {statistic}, p-value: {p_value}")

statistic, p_value = stats.kruskal(
    dist_rwd[metric],
    dist_pen[metric],
)
print(f"{metric}: Rwd vs Pen Mann-Whitney U statistic: {statistic}, p-value: {p_value}")

statistic, p_value = stats.kruskal(dist_rwd[metric], dist_pen[metric], dist_std[metric])
print(f"{metric}: Std vs Pen vs Rwd Kruskal statistic: {statistic}, p-value: {p_value}")

closeness: Std vs Rwd Mann-Whitney U statistic: 15059528.0, p-value: 0.0
closeness: Std vs Pen Mann-Whitney U statistic: 281163.0, p-value: 0.0
closeness: Rwd vs Pen Mann-Whitney U statistic: 5806.742163866546, p-value: 0.0
closeness: Std vs Pen vs Rwd Kruskal statistic: 9691.41452147344, p-value: 0.0


In [65]:
metrics_df[metrics_df["closeness"] >= 0.7].sort_values(by=["gene"]).groupby("Type").count()

,gene,betwenees,closeness,degree,strength,pageRank,hubScore,IVI,TF
Type,,,,,,,,,
Penalised,17,17,17,17,17,17,17,17,17
Reward,12,12,12,12,12,12,12,12,12
Standard,12,12,12,12,12,12,12,12,12


In [40]:
metrics = ["betwenees", "closeness", "degree", "pageRank", "IVI"]
test_rest = []

for metric in metrics:
    metric_results = []
    statistic, p_value = stats.mannwhitneyu(
        dist_std[metric],
        dist_rwd[metric],
    )
    metric_results.append((statistic, p_value))

    statistic, p_value = stats.mannwhitneyu(
        dist_std[metric],
        dist_pen[metric],
    )
    metric_results.append((statistic, p_value))

    statistic, p_value = stats.mannwhitneyu(
        dist_rwd[metric],
        dist_pen[metric],
    )
    metric_results.append((statistic, p_value))
    test_rest.append(metric_results)

pd.DataFrame(test_rest, columns=["Standard vs Reward", "Standard vs Penalised", "Reward vs Penalised"], index=metrics)

,Standard vs Reward,Standard vs Penalised,Reward vs Penalised
betwenees,"(7998031.0, 0.9847939363634195)","(8447400.0, 9.335849220751918e-09)","(8437222.5, 1.6750357375666685e-08)"
closeness,"(15059528.0, 0.0)","(281163.0, 0.0)","(93297.0, 0.0)"
degree,"(8062709.0, 0.5354114636615257)","(7864292.5, 0.981719202372621)","(7802986.0, 0.5550597787923791)"
pageRank,"(8076267.0, 0.460270769241348)","(7551878.0, 0.0023506429567561973)","(7488446.0, 0.0002481832615550817)"
IVI,"(8135410.0, 0.18985272610494086)","(6891664.5, 1.7677669311548796e-21)","(6747375.5, 8.000283094178819e-28)"


## 4K 10 TF

In [11]:
if True:
    std_nt, rwrd_nt, pen_nt = tum.exps["standard_4K_10TF"], tum.exps["norm3_4K_10TF"], tum.exps["beta_4K_10TF"]
    metrics_df = gh.prep_net_metrics(std_nt, rwrd_nt, pen_nt)
    fig = gh.plot_net_metrics(metrics_df, label="10TF", filename="NetworkMetricsComp_{}".format("10TF"), figs_path=figures_path)

## Mutation representation (report)

In [12]:
t_all_stats_df = gh.stats_mut_burden(all_tum_tpms, tf_list, tcga_mutations_df, type="All")
t_4k_stats_df = gh.stats_mut_burden(tum.exps["standard_4K_3TF"].tpm_df, tf_list, tcga_mutations_df, type="4K")

all_df = pd.concat([t_all_stats_df, t_4k_stats_df], axis=0)

In [13]:
fig = gh.plot_mut_rep(all_df, title="Tumour. 4K top varied vs All expressed.")
fig = fig.update_layout(
    title="",
    legend=dict(
        orientation="h",
        title="Genes included",
        yanchor="bottom",
        xanchor="center",
        y=0.9,
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=22, color="#003366"),
    ),
    xaxis=dict(tickfont=dict(size=18), title="Mutation burden"),
    yaxis=dict(tickfont=dict(size=18)),
    font=dict(size=18),
)
fig.update_yaxes(matches=None)
fig.show()
save_fig(name="MutTF_representation_4K-all", fig=fig, base_path=figures_path, width=1400, height=700, margin=0.02)

In [14]:
mut_tf = all_df[(all_df.index.str.contains("TF")) & (all_df["Type"] == "4K")]
mut_tf["Prct"] = round(mut_tf["Num"] / 265, 2)

sel_4k_df = all_df[(~all_df.index.str.contains("TF")) & (all_df["Type"] == "4K")]
sel_4k_df["Prct"] = round(sel_4k_df["Num"] / 4000, 2)

comb_df = pd.concat([mut_tf, sel_4k_df], axis=0).reset_index(names="Gene Type")
comb_df.loc[comb_df["Gene Type"] == "Mut", "Gene Type"] = "4K"
comb_df.loc[comb_df["Gene Type"] == "Mut_TF", "Gene Type"] = "TF"

fig = px.bar(
    comb_df,
    x="Burden",
    # y="Prct",
    y="Prct",
    color="Gene Type",
    barmode="group",
    text_auto=True,
    height=700,
)

fig = fig.update_layout(
    title="",
    legend=dict(
        orientation="h",
        title="Genes included",
        yanchor="bottom",
        xanchor="center",
        y=0.9,
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=22, color="#003366"),
    ),
    xaxis=dict(tickfont=dict(size=18), title="Mutation burden"),
    yaxis=dict(tickfont=dict(size=18), title="Ratio"),
    font=dict(size=18),
)
fig.show()
save_fig(name="MutTF_representation_4K_TF_prct", fig=fig, base_path=figures_path, width=1400, height=500, margin=0.02)

# TF 6 (report)

## Comp

### Norm3 and standard

In [15]:
%autoreload 2
show_figures = False

std_norm3_comp_tf10 = NetworkComp(tum, 4, "standard_4K_6TF", "norm3_4K_6TF")
std_norm3_comp_tf10.diff_in_com()

if 0:
    comp_dict = std_norm3_comp_tf10.comp_ge_comm()
    if False:
        # TODO move this into NetworkComp
        for key, df in comp_dict.items():
            fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum Median values in communities for {}".format(key), points="all")
            fig.show()

        display(std_norm3_comp_tf10.sankey_plot())
        display(std_norm3_comp_tf10.com_mut_distrib(include_source=False))
        display(std_norm3_comp_tf10.membership_change(include_source=False))

if show_figures:
    dmy_df, meta_norm3 = std_norm3_comp_tf10.comb_mut_stats(direction="Target")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Target"))
    display(NetworkComp.plot_corr_matrix_coms(meta_norm3, height=700, title="Tum derived. Corr matrix for standard", hide_up=True))

    dmy_df, meta_std = std_norm3_comp_tf10.comb_mut_stats(direction="Source")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Source"))
    display(NetworkComp.plot_corr_matrix_coms(meta_std, height=700, title="Tum derived. Corr matrix for reward", hide_up=True))

norm3_4K_6TF has the following communities in addition to standard_4K_6TF: {2, 19, 21, 15}


## Cluster methods

In [16]:
tf = 6
comb_std, _, _ = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _ = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf), show_figs=False)
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _ = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf6 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

Variation per principal component [0.33414667 0.26297279] and the sum 59.71%
Variation per principal component [0.34480403 0.26605455] and the sum 61.09%
Variation per principal component [0.34948211 0.24750797] and the sum 59.70%


In [17]:
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "TCGA408_classifier",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
    "2019_consensus_classifier",
]
sky.main(df=comb_tf6, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

## Modularity score and Sankey (Report)

In [18]:
tf, no_K, no_genes, cs_model = 6, 6, "4K", "RawKMeans"

tum, sky_fig, cols = gh.prep_sankey_leiden(tum, vu_output=vu_output, tf=tf, no_K=no_K, no_genes=no_genes, chosen_cs_model=cs_model)
sky_fig.update_layout(
    title="",
    font=dict(size=18),
)
leiden_sky = gh.plot_sankey_leiden(tum, sky_fig=sky_fig, rename_cols=cols, label="TF-{}".format(tf), tf=str(tf))


leiden_sky = leiden_sky.update_layout(title="", xaxis=dict(tickfont=dict(size=18)), yaxis=dict(tickfont=dict(size=18)), font=dict(size=18), height=900)
leiden_sky = leiden_sky.update_annotations(font_size=22)
save_fig(name="LeidenMetrics_Sankey_{}".format("TF-6"), fig=leiden_sky, base_path=figures_path, width=1400, height=1000)

Variation per principal component [0.33414667 0.26297279] and the sum 59.71%
Variation per principal component [0.34480403 0.26605455] and the sum 61.09%
Variation per principal component [0.34948211 0.24750797] and the sum 59.70%


## Stats

In [30]:
_, all_leiden = tum.avg_leiden_scores()

# filtering an pre-processing
all_leiden = all_leiden.loc[all_leiden["TF"] == "6"]
all_leiden.loc[all_leiden["Modifier"] == "beta", "Modifier"] = "Penalised"

In [31]:
leid_std = all_leiden.loc[all_leiden["Modifier"] == "Standard"]
leid_rwd = all_leiden.loc[all_leiden["Modifier"] == "Reward"]
leid_pen = all_leiden.loc[all_leiden["Modifier"] == "Penalised"]

metric = "ModularityScore"
statistic, p_value = stats.kruskal(leid_std[metric], leid_pen[metric], leid_rwd[metric])
print(f"{metric}: Std vs Pen vs Rwd Kruskal statistic: {statistic}, p-value: {p_value}")

metric = "ModuleNum"
statistic, p_value = stats.kruskal(leid_std[metric], leid_pen[metric], leid_rwd[metric])
print(f"{metric}: Std vs Rwd Kruskall statistic: {statistic}, p-value: {p_value}")

ModularityScore: Std vs Pen vs Rwd Kruskal statistic: 25.80645161290323, p-value: 2.49000508218493e-06
ModuleNum: Std vs Rwd Kruskall statistic: 3.304781453755149, p-value: 0.1915913181430947


In [39]:
metrics = ["ModularityScore", "ModuleNum"]
test_rest = []

for metric in metrics:
    metric_results = []
    statistic, p_value = stats.mannwhitneyu(
        leid_std[metric],
        leid_rwd[metric],
    )
    metric_results.append((statistic, p_value))

    statistic, p_value = stats.mannwhitneyu(
        leid_std[metric],
        leid_pen[metric],
    )
    metric_results.append((statistic, p_value))

    statistic, p_value = stats.mannwhitneyu(
        leid_pen[metric],
        leid_rwd[metric],
    )
    metric_results.append((statistic, p_value))
    test_rest.append(metric_results)

pd.DataFrame(test_rest, columns=["Standard vs Reward", "Standard vs Penalised", "Reward vs Penalised"], index=metrics).T

,ModularityScore,ModuleNum
Standard vs Reward,"(0.0, 0.00018267179110955002)","(68.5, 0.15625270616013115)"
Standard vs Penalised,"(100.0, 0.00018267179110955002)","(59.0, 0.5057632526965341)"
Reward vs Penalised,"(0.0, 0.00018267179110955002)","(70.0, 0.10914769693061342)"


In [37]:
metric = "ModularityScore"
statistic, p_value = stats.kruskal(leid_std[metric], leid_pen[metric])
print(f"{metric}: Std vs Pen Kruskal statistic: {statistic}, p-value: {p_value}")

statistic, p_value = stats.kruskal(leid_std[metric], leid_rwd[metric])
print(f"{metric}: Std vs Rwd Mann statistic: {statistic}, p-value: {p_value}")

statistic, p_value = stats.kruskal(leid_rwd[metric], leid_pen[metric])
print(f"{metric}: Rwd vs Pen Kruskal statistic: {statistic}, p-value: {p_value}")

ModularityScore: Std vs Pen Kruskal statistic: 14.285714285714278, p-value: 0.0001570522842307523
ModularityScore: Std vs Rwd Mann statistic: 14.285714285714278, p-value: 0.0001570522842307523
ModularityScore: Rwd vs Pen Kruskal statistic: 14.285714285714278, p-value: 0.0001570522842307523


# TF 10

Experiments with lower numbers of edges per TF

## Std vs Norm3 (Reward v2)

In [19]:
%autoreload 2
std_norm3_comp_tf = NetworkComp(tum, 4, "standard_4K_10TF", "norm3_4K_10TF")
std_norm3_comp_tf.diff_in_com()

show_figures = False

if show_figures:
    comp_dict = std_norm3_comp_tf.comp_ge_comm()
    for key, df in comp_dict.items():
        fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum Median values in communities for {}".format(key), points="all")
        # fig.show()

    # display(std_norm3_comp_tf.sankey_plot())
    display(std_norm3_comp_tf.com_mut_distrib(include_source=True, binarySource=True))
    # display(std_norm3_comp_tf.membership_change(include_source=False))


if show_figures:
    dmy_df, meta_norm3 = std_norm3_comp_tf.comb_mut_stats(direction="Target")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Target"))
    display(NetworkComp.plot_corr_matrix_coms(meta_norm3, height=700, title="Tum derived. Corr matrix for standard", hide_up=True))

    dmy_df, meta_std = std_norm3_comp_tf.comb_mut_stats(direction="Source")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Source"))
    display(NetworkComp.plot_corr_matrix_coms(meta_std, height=700, title="Tum derived. Corr matrix for reward", hide_up=True))

norm3_4K_10TF has the following communities in addition to standard_4K_10TF: {16, 20, 21, 23}


In [20]:
selModCon = tum.exps["norm3_4K_10TF"].modCons
for com in [16, 20, 21, 23]:
    print("* Com = {}".format(com))
    print(selModCon[com].index.values)
    # print("* Com_{} = {}".format(com, ", ".join(selModCon[com].index.values)))

* Com = 16
['DEDD' 'ATF6' 'USP21' 'B4GALT3' 'UHMK1' 'UAP1' 'PEX19' 'DUSP12' 'ALDH9A1'
 'PPOX' 'KIFAP3' 'MPZL1' 'F11R' 'EEF1AKNMT' 'IGSF8' 'TSTD1' 'RP11-297K8.2'
 'TOMM40L' 'KLHDC9' 'IGSF9' 'PIGK' 'CREG1' 'MPC2' 'DUSP23' 'TSEN15'
 'HSPA6']
* Com = 20
['YES1' 'USP14' 'AFG3L2' 'RAB12' 'SEH1L' 'CHMP1B' 'PPP4R1' 'SMCHD1'
 'MPPE1' 'IMPA2' 'CHMP1B-AS1' 'TWSG1' 'TTC39C' 'NCK2' 'CABYR']
* Com = 21
['P4HTM' 'NPRL2' 'ABHD14A' 'NT5DC2' 'PCBP4' 'CYB561D2' 'ABHD14B' 'ACY1'
 'TREX1' 'SMIM4' 'CHCHD10' 'SPATC1L']
* Com = 23
['ID3' 'ID2' 'ID4' 'DUSP2' 'SMAD6' 'IGFBP2' 'BMP2']


### Mutation evolution

In [21]:
if 0:
    dmy_df = std_norm3_comp_tf.comb_mut_stats(direction="Target")
    display(std_norm3_comp_tf.plot_mut_evo(dmy_df, direction="Target"))

    dmy_df = std_norm3_comp_tf.comb_mut_stats(direction="Source")
    display(std_norm3_comp_tf.plot_mut_evo(dmy_df, direction="Source"))

## Std vs beta

In [22]:
%autoreload 2
std_beta_comp_tf = NetworkComp(tum, 4, "standard_4K_10TF", "beta_4K_10TF")
std_beta_comp_tf.diff_in_com()

show_figures = False
if show_figures:
    comp_dict = std_beta_comp_tf.comp_ge_comm()
    for key, df in comp_dict.items():
        fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum Median values in communities for {}".format(key), points="all")
        # fig.show()


    # display(std_beta_comp_tf.sankey_plot())
    display(std_beta_comp_tf.com_mut_distrib(include_source=True,  binarySource=True))
    # display(std_beta_comp_tf.membership_change(include_source=False))


if show_figures:
    std_beta_comp_tf.comp_df = std_beta_comp_tf.comp_df.fillna(0)
    dmy_df, meta_std = std_beta_comp_tf.comb_mut_stats(direction="Target")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Target"))
    display(NetworkComp.plot_corr_matrix_coms(meta_std, height=700, title="Tum derived. Corr matrix for standard", hide_up=True))

    dmy_df, meta_beta = std_beta_comp_tf.comb_mut_stats(direction="Source")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Source"))
    display(NetworkComp.plot_corr_matrix_coms(meta_beta, height=700, title="Tum derived. Corr matrix for beta", hide_up=True))

beta_4K_10TF has the following communities in addition to standard_4K_10TF: {11, 13, 14, 18, 20, 22}


## Morpheus
### Export

In [23]:
%autoreload 2 
if 0:
    no_genes = 100
    for exp in tum.get_exps():
        if "beta2" in exp.name or "beta3" in exp.name:
            continue

        sort_col = "ModCon_{}".format(exp.type)

        exp.mevsMut, data = exp.get_mevs(exp.tpm_df, exp.modCons, sort_col=sort_col, num_genes=100)
        exp.export_morpheus_mevs(vu_output, exp.name + "_mut_100", tum=True)

### Import

In [24]:
def import_morpheus(path, col_name="PGCNA_cut"):
    df = pd.read_csv(path, sep="\t", skiprows=2, index_col="id")
    df.index.names = ["Sample"]
    df = df.transpose().rename(columns={"dendrogram_cut": col_name})
    return df

In [25]:
if 0:
    m_path = "{}/Stats/morpheus/10TF/cut_{}/".format(tum.path, 6)

    morpheus_std = import_morpheus(path="{}/{}_g{}.gct".format(m_path, "std", 100), col_name="tum_std")
    morpheus_norm3 = import_morpheus(path="{}/{}_g{}.gct".format(m_path, "norm3", 100), col_name="tum_norm3")
    morpheus_beta = import_morpheus(path="{}/{}_g{}.gct".format(m_path, "beta", 100), col_name="tum_beta")
    dmy_df = pd.concat([morpheus_std["tum_std"], morpheus_norm3["tum_norm3"], morpheus_beta["tum_beta"], vu_output], axis=1).dropna()
    # dmy_df.drop(index=["dendrogram_cut"], axis=1, inplace=True)
    reorder_cols = [
        "KMeans_labels_5",
        "TCGA408_classifier",
        "tum_std",
        "tum_norm3",
        "tum_beta",
    ]
    sky.main(df=dmy_df, reorder_cols=reorder_cols, title="Cut 7, Comparison between " + ", ".join(reorder_cols))

    sel_samples = dmy_df[dmy_df["TCGA"] == "Luminal_infiltrated"].copy(deep=True)
    sel_samples = dmy_df.copy(deep=True)
    sel_samples["mut_count"] = tcga_mutations_df.sum(axis=0)

    px.histogram(sel_samples, x="TCGA408_classifier", y="mut_count")

## Clustering methods

Find the optimal clustering

### Modifiers - best results

* Standard
  * Best results are given by clusters 4-6

* Norm3
  * When I run the experimnts with clustering methods: `["Birch", "RawKMeans", "GaussianMixture", "Ward", "SpectralClustering", "Avg"]` and comparted the metrics the Kmeans 4-6 exhibited the best results
* Beta

In [26]:
tf = 10
comb_std, _, _ = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _ = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf), show_figs=False)
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _ = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf10 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

Variation per principal component [0.34775526 0.26150788] and the sum 60.93%
Variation per principal component [0.3370373  0.27936119] and the sum 61.64%
Variation per principal component [0.34907942 0.25774995] and the sum 60.68%


### Compare the best results between norm3 and standard

On K-means comparisons:
 * From what I can see that there is no major difference when CS is set between 3-4, but from >5 there are large differences.

Avg:
* Can't make the difference at Basal split

Spectral Clustering 
* It can only split the Basal after CS > 7
* Apart from that it has a similar behaviour with Kmeans


In [27]:
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "KMeans_labels_5",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
]
# sky.main(df=comb_tf10, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

# variations = ["_".join(col.split("_")[:2]) for col in comb_std.columns]
# set(variations)

# TF 50

## Median values for each community

In [28]:
%autoreload 2
std_norm3_comp_tf50 = NetworkComp(tum, 4, "standard_4K_50TF", "norm3_4K_50TF")
std_norm3_comp_tf50.diff_in_com()
display(std_norm3_comp_tf50.sankey_plot())

if 0:
    comp_dict = std_norm3_comp_tf50.comp_ge_comm()
    for key, df in comp_dict.items():
        fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum. Median values in communities for {}".format(key), points="all")
        fig.show()

    display(std_norm3_comp_tf50.sankey_plot())
    display(std_norm3_comp_tf50.com_mut_distrib(include_source=True))
    display(std_norm3_comp_tf50.membership_change(include_source=True))

norm3_4K_50TF has the following communities in addition to standard_4K_50TF: {5}


## Mutation evolution


In [29]:
if 0:
    dmy_df = std_norm3_comp_tf50.comb_mut_stats(direction="Target")
    display(std_norm3_comp_tf50.plot_mut_evo(dmy_df, direction="Target"))

    dmy_df = std_norm3_comp_tf50.comb_mut_stats(direction="Source")
    std_norm3_comp_tf50.plot_mut_evo(dmy_df, direction="Source")

## Clustering

In [30]:
tf = 50
comb_std, _, _ = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _ = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf))
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _ = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf50 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

Variation per principal component [0.34623    0.28250341] and the sum 62.87%
Variation per principal component [0.38524433 0.2417056 ] and the sum 62.69%
Variation per principal component [0.41105467 0.20591684] and the sum 61.70%


In [31]:
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "KMeans_labels_5",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
]
# sky.main(df=comb_tf50, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

# TF 3

## Cluster methods

In [32]:
tf = 3
comb_std, _, _ = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _ = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf))
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _ = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf3 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

Variation per principal component [0.324855   0.27349363] and the sum 59.83%
Variation per principal component [0.34426125 0.25185594] and the sum 59.61%
Variation per principal component [0.35032186 0.24843387] and the sum 59.88%


In [33]:
num = 3
cluster_model = "RawKMeans"
reorder_cols = [
    "KMeans_labels_5",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
]
# sky.main(df=comb_tf3, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

# Clutering comp across TF experiments

## Norm3 (Reward)

In [34]:
comb_all_tfs = pd.concat(
    [comb_tf3, comb_tf6.drop(columns=vu_output.columns), comb_tf10.drop(columns=vu_output.columns), comb_tf50.drop(columns=vu_output.columns)], axis=1
)

reorder_cols = [
    "KMeans_labels_6",
    # "{}_CS_{}_std_tf{}".format(cluster_model, num, 6),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 3),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 6),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10),
    "RawKMeans_CS_6",
    # "KMeans_labels_5",
    # "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 50),
]
sky.main(df=comb_all_tfs, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format("Norm3", ", ".join(reorder_cols)))

## TF 10 norm3 and standard

In [35]:
comb_all_tfs = pd.concat([comb_tf6.drop(columns=vu_output.columns), comb_tf10, comb_tf50.drop(columns=vu_output.columns)], axis=1)
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "TCGA408_classifier",
    "KMeans_labels_6",
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10),
    "{}_CS_{}_std_tf{}".format(cluster_model, num, 10),
    # "RawKMeans_CS_6",
    # "KMeans_labels_5",
    # "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 50),
]
# for col in reorder_cols:
#     comb_all_tfs[col] = comb_all_tfs[col].astype(str)
title = "Bio+Kmeans_5 - Network_Standard - Network_Reward - TCGA"
meta, fig = sky.main(df=comb_all_tfs, reorder_cols=reorder_cols, title=title, retMeta=True)
fig.update_layout(height=700)
fig.show()

## Exploring Community enrichment for Basal 3 splits

In [37]:
transform_cols = ["TCGA408_classifier", "consensus", "Lund2017.subtype"]
for col in transform_cols:
    vu_output[col] = vu_output[col].astype("category")
    vu_output[col + "_num"] = vu_output[col].cat.codes

In [38]:
cluster_model = "RawKMeans"
num = 6
sel_exp = tum.exps["norm3_4K_10TF"]
sel_model = "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10)
prep_df = comb_tf10.loc[comb_tf10[sel_model].isin([2.0, 5.0, 4.0])]  # select the inf enriched clusters

sel_cols = ["KMeans_labels_6", sel_model]
prep_df = prep_df[sel_cols]
prep_df["KMeans_labels_6_num"] = pd.factorize(prep_df["KMeans_labels_6"])[0]
prep_df.drop(columns=["KMeans_labels_6"], inplace=True)

prep_df = pd.concat(
    [
        prep_df,
        sel_exp.mevsMut,
    ],
    axis=1,
).dropna()


mevs_name = "mevs_{}.tsv".format("Basal3_norm3")
prep_df.transpose().to_csv("{}stats/{}".format(sel_exp.exps_path, mevs_name), sep="\t", index=True)

In [39]:
comb_norm3 = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf))

Variation per principal component [0.34426125 0.25185594] and the sum 59.61%


In [40]:
dmy_df = comb_tf10.loc[comb_tf10[sel_model].isin([2.0, 5.0, 4.0])]  # select the inf enriched clusters
dmy_df = pd.concat([dmy_df[sel_model], sel_exp.mevsMut], axis=1).dropna()

dmy_df = dmy_df.loc[dmy_df[sel_model] == 2.0][dmy_df.columns[1:]]
top_coms = {}
num_top = 10
cmn_coms = set()
for idx, value in dmy_df.iterrows():
    cmn_comns = set(value.sort_values(ascending=False)[:num_top].index.values) & set(cmn_coms)

### Estimate score

In [43]:
cluster = "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10)
comb_all_tfs[cluster] = comb_all_tfs[cluster].astype(str)
# comb_all_tfs.loc[((comb_all_tfs["KMeans_labels_6"] == "High IFNG") & (comb_all_tfs[cluster] == "2.0")), cluster ]  = "inf"
fig1 = cs.plot_meta_scores(comb_all_tfs, y_axis="IFNG_score", classification=cluster, size="infiltration_score")

fig1.update_layout(
    legend=dict(
        orientation="v",
        title="Network_KMeans6",
        yanchor="middle",
        # y=0.74,
        xanchor="right",
        # x=0.13,
        bgcolor="rgba(0,0,0,0)",
    ),
    height=500,
)
# meta, fig = sky.main(df=comb_all_tfs, reorder_cols=reorder_cols, title=title, retMeta=True)
# fig.show()
# fig1.show()

# Comparing between experiments

Curious to see what's the difference between the 2 modifiers. If Beta is better at separating the communities.

Note: I couldn't see too much of a change when beta is applied. In fact, it feels that beta is very similar to the norm3 modifier

In [46]:
# Prep data
std_t = tum.exps["standard_4K_10TF"]
norm3_t = tum.exps["norm3_4K_10TF"]
beta_t = tum.exps["beta_4K_10TF"]

dmy_std = std_t.nodes_df.rename(columns={"Modularity Class": "std_4K_10TF"})
dmy_norm3 = norm3_t.nodes_df.rename(columns={"Modularity Class": "norm3_4K_10TF"})
dmy_beta = beta_t.nodes_df.rename(columns={"Modularity Class": "beta_4K_10TF"})

# combine data
test = pd.concat([dmy_std["std_4K_10TF"], dmy_beta["beta_4K_10TF"], dmy_norm3["norm3_4K_10TF"]], axis=1).fillna(-1)  # dmy_h3["norm3_h_5k"]], axis=1)
reoder_cols = ["std_4K_10TF", "norm3_4K_10TF", "beta_4K_10TF"]  # "norm3_h_5k"]

# plot
meta = sky.main(
    test,
    reorder_cols=reoder_cols,
    title="Community comparison between " + ", ".join(reoder_cols),
)

In [47]:
sel_mut = tcga_mutations_df[(tcga_mutations_df.index.isin(norm3_t.tpm_df.index)) & (tcga_mutations_df["count"] > 2)]
print("Genes mutated", sel_mut.shape[0] / norm3_t.tpm_df.shape[0])

Genes mutated 0.532


In [48]:
community_sizes = test["norm3_4K_10TF"].value_counts()
small_coms = community_sizes[community_sizes < 100].index.values
small_coms = test[test["norm3_4K_10TF"].isin(small_coms)]
# dmy_df = test.copy(deep=True)
# dmy_df.loc[~dmy_df["norm3_4K_10TF"].isin(small_coms), "norm3_4K_10TF"] = -1

meta = sky.main(
    small_coms,
    reorder_cols=reoder_cols,
    title="Just small communities comparison between " + ", ".join(reoder_cols),
)